# Libraries

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

## Visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt

## Hypothesis testing
from scipy.stats import (ttest_ind, f_oneway, shapiro, levene, chi2_contingency)
from statsmodels.stats.proportion import proportions_ztest # For proportion z test
from statsmodels.stats.anova import anova_lm # For n-way anova
from statsmodels.formula.api import ols # For n-way anova
from scipy import stats
import statsmodels.api as sm ## For QQ plot

## Modelling
from sklearn.model_selection import (train_test_split,KFold,cross_val_score,GridSearchCV)
from sklearn.preprocessing import (StandardScaler,MinMaxScaler)
from sklearn.metrics import (mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error
                             accuracy_score,confusion_matrix,
                             roc_auc_score,roc_curve,auc,
                             precision_recall_curve,classification_report)
from sklearn.impute import KNNImputer
from sklearn.linear_model import (Lasso,Ridge,LinearRegression,LogisticRegression)
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE

import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from cmfrec import CMF
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

##Generic
import re
import os
import random
import time
import warnings
warnings.filterwarnings("ignore")

# Reading data and creating dataframe

In [ ]:
df=pd.read_csv("netflix.csv")

import gdown
url='https://drive.google.com/file/d/1gHYYLqLt6rMyeAyvHf1wvlQ4BLKwjv9W/view?usp=sharing'
ider=url.split('/')[-2]
!gdown --id $ider
train = pd.read_csv('/content/train_1.csv')

# Examining dataset

In [ ]:
df.head()
df.shape
df.dtypes
df.info()
df.columns
df.select_dtypes('object').columns
df.select_dtypes(np.number).columns
num_cols=data.select_dtypes(include=np.number).columns

df.isnull().sum()/len(df)*100
df.isna().sum()

df['Miles'].min(), df['Miles'].max()

df.sort_values(by="release_year").head(20)
df_new=new_train.sort_index( ascending=[True,True])  ## If there are multiple indexes

df["director"].value_counts()[:10]
data['zip_code'].value_counts(normalize=True)*100
df['company_hash'].value_counts().sort_index()

 ## To get descriptive statistics
df['Income'].describe()
df.describe()
df.describe(include='all').T

# Exploratory data analysis

In [ ]:
##Date columns
data['MMM-YY']=pd.to_datetime(data['MMM-YY'])

## Column operations
movies.rename(columns={'Movie ID':'MovieID'}, inplace=True)
movies1=movies.copy()

data['Rating']=data['Rating'].astype('int32')

## Creating years of experience column
df['years_of_experience']=2024-df['orgyear']

## Unique Values
df['rating'].unique()   # <=To get list of unique values
print(df.title.nunique() == df.shape[0])

## Missing values
df['date_added'].isna().sum()
df[df['date_added'].isna()]
##Sometimes we need to extract year from date . More than exact date, growth over years is significant.

## Filling null values
df['orgyear'].fillna(df.groupby('company_hash')['orgyear'].transform('median'), inplace=True)
df['job_position'] = df['job_position'].fillna('Others')

df = df.loc[~df['orgyear'].isna()]

## Dropping null values
df.dropna(inplace=True)

## Filtering
country_list=["United States","India","United kingdom","Canada","Japan"]
df2=df1[df1["country"].isin(country_list)]

## Working with data
x=df['cast'].apply(lambda x: str(x).split(', ')).tolist()
df_new=pd.DataFrame(x,index=df['title'])

df.reset_index(inplace=True)
df.set_index('index', inplace=True)

def preprocess_string(string):
    new_string= re.sub('[^A-Za-z ]+', '', string).lower().strip()
    return new_string

df.job_position=df.job_position.apply(lambda x: preprocess_string(str(x)))

## Grouping
df.groupby('Gender')['User_ID'].nunique()
df.groupby('Gender')['Purchase'].describe()
df.sample(300).groupby('Gender')['Purchase'].describe()
data.groupby(by=['grade','loan_status']).size()
df.groupby(['region','sex','smoker'])['hospitalization charges'].mean().unstack()

function_dict = {'Age':'max', 'Gender':'first','City':'first',
 'Education_Level':'last', 'Income':'last'}
new_train=df.groupby(['Driver_ID','MMM-YY']).aggregate(function_dict)
new_train.head(10)

df1['Gender'] = list(df_new.groupby('Driver_ID').agg({'Gender':'last'})['Gender'])

## Dropping duplicates,columns
df.drop_duplicates(inplace=True)
df.drop(columns='cast_x', inplace=True)
df.drop(["Unnamed: 0"],axis=1,inplace=True)

## Outlier treatment
percentiles = df[col].quantile([0.05,0.95]).values
df[col] = np.clip(df[col], percentiles[0], percentiles[1])

df['ctc'] = df['ctc'].clip(lower=df.ctc.quantile(0.01), upper=df.ctc.quantile(0.99))

q1=df['Income'].quantile(0.25)
q3=df['Income'].quantile(0.75)
iqr=q3-q1
df=df.loc[(df['Income']>(q1-1.5*iqr)) & (df['Income']<(q3+1.5*iqr))]

## Binning data
bins = [-1,20,25,30,35,40,55]
labels = ['<20','20-25','25-30','30-35','35-40','40+']
df['Age_bins'] = pd.cut(df['Age'], bins=bins, labels=labels)

## Merging
final=df.merge(df_new,on='title',how='inner')

## Combining minority classes to a single one
data.loc[(data["home_ownership"]=="ANY") | (data["home_ownership"]=="NONE"),"home_ownership"]="OTHER"

## Creating a balanced dataset
df_new['workingday'].value_counts()
working=df_new[df_new['workingday']==1]['count'].sample(3425) ## 3425 is the minimum value in the value_counts output
nonworking=df_new[df_new['workingday']==0]['count'].sample(3425)

## Mapping of target variable
data['loan_status']=data['loan_status'].map({'Fully Paid':0,'Charged Off':1})

## Applying function
def fun(number):
  if number==0.0:
    return 0
  else:
    return 1

data['pub_rec']=data['pub_rec'].apply(fun)

data['zip_code']=data['address'].apply(lambda x:x[-5:]) ## Extracting zip code from address

## Correlation between columns. First convert categorical to numerical
df['Gender'].replace(['Male', 'Female'], [1, 0], inplace=True)
df.corr()

## For probabilities (conditional,marginal)
pd.crosstab(index=df['Gender'],columns=df['Product'])
pd.crosstab(index=df['Gender'],columns=df['Product'],margins=True)
pd.crosstab(index=df['Gender'],columns=df['Product'],margins=True,normalize=True)*100
pd.crosstab(index=df['Gender'],columns=df['Product'],margins=True,normalize='index')

## Mean target imputation
total_acc_avg=data.groupby(by='total_acc').mean(numeric_only=True)['mort_acc']

def fill_mort_acc(total_acc,mort_acc):
  if np.isnan(mort_acc):
    return total_acc_avg[total_acc].round()
  else:
    return mort_acc

data['mort_acc']=data.apply(lambda x: fill_mort_acc(x['total_acc'],x['mort_acc']),axis=1)

##KNN imputation
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(data_nums)
data_new=imputer.transform(data_nums)
data_new=pd.DataFrame(data_new)
data_new.columns=cols
data_new.head()

## One hot encoding
dummies=['purpose','zip_code','home_ownership','grade','verification_status','application_type']
data=pd.get_dummies(data,columns=dummies,drop_first=True)

df1 = pd.concat([df1,pd.get_dummies(df1['City'],prefix='City')],axis=1)
df1.head()

##Generic
rem_cols=list(set(data.columns).difference(set(cols)))

# Visual analysis

In [ ]:
df['type'].value_counts().plot(kind='pie')
plt.pie(df["Product"].value_counts(),labels=["KP281","KP481","KP781"],autopct="%0f%%")
plt.show()

## Plotting correlation
sns.heatmap(df.corr(), cmap="YlGnBu", annot=True)
sns.heatmap(data.corr(numeric_only=True),annot=True,cmap='Greens')

## For identifying outliers
sns.boxplot(x=df["Age"])

plt.figure(figsize=(15,6))
for i,j in enumerate(categorical):
  plt.subplot(1,4,i+1)
  plt.subplots_adjust(hspace=0.8)
  sns.boxplot(x=j,y='count',data=df)
  plt.tight_layout(pad=1)

## Multivariate analysis using box plot
sns.boxplot(x="Product", y="Usage", hue="Gender", data=df)
x=df[df['Gender']=="Male"]
print(x[x["Product"]=="KP481"]["Usage"].value_counts())

## Count plot
sns.countplot(x="Product", hue="MaritalStatus", data=df)

## Violin plot
sns.violinplot(x="Fitness", y="Age", data=df)

## Line plot
sns.lineplot(x='age',y='hospitalization charges',data=df,hue='sex')

##Bar plot
sns.barplot(x='age_bins',y='hospitalization charges',data=df)

plt.subplot(231)
df1['Grade'].value_counts(normalize=True).plot.bar(title='Grade')

## To observe association between 2 variables
plt.scatter(df['Age'], df['Income'],linewidths = 2,marker ="o",edgecolor ="green",s = 50)
sns.scatterplot(x="CGPA", y="LOR ", data=df, hue="Research")

sns.regplot(x="GRE Score", y="TOEFL Score", data=df)

## Analysis of categorical columns
sns.distplot(df['Age'], hist=True, kde=True, bins=int(36))  ## Binning age and plotting distribution

sns.displot(x='Purchase',data=df,bins=25,hue='Gender')
plt.xlabel('Purchase')
plt.ylabel('Frequency')
plt.title('Histogram of purchase')

sns.histplot(data=df,x="release_year",hue="type")

# Hypothesis Testing

In [ ]:
## T-test independent
t_statistic,p_value=ttest_ind(working,nonworking,equal_var=False,alternative='greater')

## Shapiro test
statistics,p_value=shapiro(df)

## Levene test
statistics,p_value=levene(df1,df2,df3)

## One way ANOVA
t_stats,p_value=f_oneway(df1,df2,df3)

## Contigency table
contigency=pd.crosstab(df_new['season'],df_new['weather'])
contigency.plot(kind='bar')
#chi2 contigency test
chi2,pval,dof,exp_freq=chi2_contingency(contigency,correction=False)

## QQ plot check
import statsmodels.api as sm
df_std=(df_female_severe['viral load']-df_female_severe['viral load'].mean())
sm.qqplot(df_std,line='45')
plt.show()


# Supervised Learning

In [ ]:
## Train test split
X = df.drop(['Chance of Admit '], axis=1)
y = df['Chance of Admit ']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.20, shuffle=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,stratify=y,random_state=42)

##Scaling
std=StandardScaler()
X_train_std=std.fit_transform(X_train)

scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

## Multiple model results - Linear,Lasso,Ridge
models = [
['Linear Regression :', LinearRegression()],
['Lasso Regression :', Lasso(alpha=0.1)],
['Ridge Regression :', Ridge(alpha=1.0)]
]

for name,model in models:
  model.fit(X_train, y_train.values)
  predictions = model.predict(std.transform(X_test))
  print (name, (np.sqrt(mean_squared_error(y_test, predictions))))
  print ('Mean Absolute Error ', mean_absolute_error(y_test.values,pred) )

## Linear Regression using statsmodels
import statsmodels.api as sm
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.values, X_train).fit()
print (model.summary())

## Linear regression assumptions check
## VIF - Checking Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(dataset,col):
  dataset=dataset.drop(columns=col,axis=1)
  vif=pd.DataFrame()
  vif['features']=dataset.columns
  vif['VIF_Value']=[variance_inflation_factor(dataset.values,i) for i in range(len(dataset.columns))]
  return vif

calculate_vif(X_train_new,[])

## Mean of Residuals
residuals = y_test.values-pred
mean_residuals = np.mean(residuals)
print ("Mean of Residuals {}".format(mean_residuals))

## Test for Homoscedasticity
p = sns.scatterplot(x=pred,y=residuals)
plt.xlabel('predicted values')
plt.ylabel('Residuals')
plt.ylim(-0.4,0.4)
plt.xlim(0,1)
p = plt.title('Residuals vs fitted values plot for homoscedasticity check')

import statsmodels.stats.api as sms
from statsmodels.compat import lzip
name = ['F statistic','p-value']
test = sms.het_goldfeldquandt(residuals, X_test)
lzip(name, test)

## Normality of residuals
p = sns.distplot(residuals,kde=True)
p = plt.title('Normality of error terms/residuals')

## Logistic Regression
logreg=LogisticRegression(max_iter=1000)
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

## Performance evaluation - Confusion Matrix,Classification report,ROC-AUC Curve,Precision Recall curve, Cross validation
confusion_matrix=confusion_matrix(y_test,y_pred)
print(confusion_matrix)

print(classification_report(y_test,y_pred))

logit_roc_auc=roc_auc_score(y_test,logreg.predict(X_test))
fpr,tpr,thresholds=roc_curve(y_test,logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr,tpr,label='Logistic Regression (area = %0.2f)'%logit_roc_auc)
plt.plot([0,1], [0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

def precision_recall_curve_plot(y_test,pred_proba_c1):
  precisions,recalls,thresholds=precision_recall_curve(y_test,pred_proba_c1)
  threshold_boundary=thresholds.shape[0]
  # plot precision
  plt.plot(thresholds,precisions[0:threshold_boundary],linestyle='--',label='precision')
  # plot recall
  plt.plot(thresholds,recalls[0:threshold_boundary],label='recalls')
  start,end=plt.xlim()
  plt.xticks(np.round(np.arange(start,end,0.1),2))
  plt.xlabel('Threshold Value');
  plt.ylabel('Precision and Recall Value')
  plt.legend();
  plt.grid()
  plt.show()

precision_recall_curve_plot(y_test,logreg.predict_proba(X_test)[:,1])

X=scaler.fit_transform(X)
kfold=KFold(n_splits=5)
accuracy=np.mean(cross_val_score(logreg,X,y,cv=kfold,scoring='accuracy'))
print("Cross Validation accuracy: {:.3f}".format(accuracy))

##Random Forest Classifier
param = {'max_depth':[2,3,4], 'n_estimators':[50,100,150,200]}
random_forest = RandomForestClassifier(class_weight ='balanced')
c = GridSearchCV(random_forest,param,cv=3,scoring='f1')
c.fit(X_train,y_train)
def display(results):
 print(f'Best parameters are : {results.best_params_}')
 print(f'The score is : {results.best_score_}')
display(c)
y_pred = c.predict(X_test)

print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

## Feature importances
importances = random_forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in random_forest.estimators_], axis=0)

## Balancing Using SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

sm = SMOTE(random_state = 7)
X_train, y_train = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train == 0)))

## XGBoost classifier

my_model = xgb.XGBClassifier(class_weight ='balanced')
my_model.fit(X_train, y_train)

y_pred = my_model.predict(X_test)

print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

## Decision Tree Classifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Unsupervised Learning

In [ ]:
## Manual clustering
grouped_c_j_y = df.groupby(['years_of_experience','job_position','company_hash'])['ctc'].describe()
df_cjy=df.merge(grouped_c_j_y, on=['years_of_experience','job_position','company_hash'], how='left')
df_cjy.drop(columns=['count','mean','std','min','25%','50%','75%','max'],inplace=True)

## Hierarchical clustering
import scipy.cluster.hierarchy as sch

sample = df.sample(500)
Z = sch.linkage(sample, method='ward')

fig, ax = plt.subplots(figsize=(20, 12))
sch.dendrogram(Z, labels=sample.index, ax=ax, color_threshold=2)
plt.xticks(rotation=90)
ax.set_ylabel('distance')

## K means clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X_sc)

kmeans.cluster_centers_   ## coordinates of the cluster centers
clusters = pd.DataFrame(X_sc, columns=X.columns)
clusters['label'] = kmeans.labels_

x_axis = 'years_of_experience'
y_axis = 'class'

plt.scatter(clusters[x_axis], clusters[y_axis], c=clusters['label'], )
plt.scatter(kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:, 2], color="red", marker="X", s=100)
plt.xlabel(x_axis)
plt.ylabel(y_axis)

In [ ]:
## Pivot table creation
matrix = pd.pivot_table(data, index='UserID', columns='Title', values='Rating', aggfunc='mean')
rm = data.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)

similar_movies = matrix.corrwith(movie_rating)

item_sim = cosine_similarity(matrix.T)
item_sim_matrix = pd.DataFrame(item_sim, index=matrix.columns, columns=matrix.columns)

## Nearest Neighbors
model_knn = NearestNeighbors(metric='cosine')
model_knn.fit(matrix.T)
distances, indices = model_knn.kneighbors(matrix.T, n_neighbors= 6)
result = pd.DataFrame(indices, columns=['Title1', 'Title2', 'Title3', 'Title4', 'Title5','Title6'])

result2 = result.copy()
for i in range(1, 7):
    mov = pd.DataFrame(matrix.T.index).reset_index()
    mov = mov.rename(columns={'index':f'Title{i}'})
    result2 = pd.merge(result2, mov, on=[f'Title{i}'], how='left')
    result2 = result2.drop(f'Title{i}', axis=1)
    result2 = result2.rename(columns={'Title':f'Title{i}'})
result2.head()

## Matrix Factorization

## 1. Using cmfrec library
model = CMF(method="als", k=4, lambda_=0.1, user_bias=False, item_bias=False, verbose=False)
model.fit(user_itm) #Fitting the model
model.A_.shape, model.B_.shape #model.A_ gives the embeddings of Users and model.B_ gives the embeddings of Items.
user_itm.Rating.mean(), model.glob_mean_  # Average rating and Global Mean
rm__ = np.dot(model.A_, model.B_.T) + model.glob_mean_ #Calculating the predicted ratings

In [ ]:
## Dickey-Fuller test - For checking stationarity

# Null Hypothesis (H0): Series is non-stationary
# Alternate Hypothesis (HA): Series is stationary
# p-value >0.05 Fail to reject (H0)
# p-value <= 0.05 Accept (H1)

def df_test(x):
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])

df_test(total_view['en'])

## Remove trend and seasonality with decomposition
decomposition = seasonal_decompose(ts.values, model='multiplicative',period=7)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plot.figure(figsize=(10,7))
plot.subplot(411)
plot.title('Observed = Trend + Seasonality + Residuals')
plot.plot(ts.values,label='Observed')
plot.legend(loc='best')
plot.subplot(412)
plot.plot(trend, label='Trend')
plot.legend(loc='best')
plot.subplot(413)
plot.plot(seasonal,label='Seasonality')
plot.legend(loc='best')
plot.subplot(414)
plot.plot(residual, label='Residuals')
plot.legend(loc='best')
plot.tight_layout()
plot.show()

ts_decompose=pd.DataFrame(residual).fillna(0)[0].values
df_test(ts_decompose)

## Remove trend and seasonality with differencing
ts_diff = ts - ts.shift(1)
plot.plot(ts_diff.values)
plot.show()
ts_diff.dropna(inplace=True)
df_test(ts_diff)

## Plot the autocorrelation and partial auto correlation functions
acf=plot_acf(ts_diff,lags=20)
pacf=plot_pacf(ts_diff,lags=20)

## ARIMA model
model = ARIMA(ts, order=(4,1,3))
model_fit = model.fit()
model_fit.predict(dynamic=False)
plot.show()

## Multistep forecasting
train = ts[:-20]
test = ts[-20:]
model = ARIMA(train, order=(4, 1, 3))
fitted = model.fit()
fc=fitted.forecast(20, alpha=0.02)
fc_series = pd.Series(fc, index=test.index)

plot.figure(figsize=(12,5), dpi=100)
plot.plot(train, label='training')
plot.plot(test, label='actual')
plot.plot(fc_series, label='forecast')
plot.title('Forecast vs Actuals')
plot.legend(loc='upper left', fontsize=8)

mape = np.mean(np.abs(fc - test.values)/np.abs(test.values))
rmse = np.mean((fc - test.values)**2)**.5

##SARIMAX
exog=ex_df['Exog'].to_numpy()
train=ts[:520]
test=ts[520:]
model=sm.tsa.statespace.SARIMAX(train,order=(4, 1, 3),seasonal_order=(1,1,1,7),exog=exog[:520])
results=model.fit()

fc=results.forecast(30,dynamic=True,exog=pd.DataFrame(exog[520:]))